In [1]:
import datetime as dt
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify

In [2]:
#################################################
# Database Setup
#################################################
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save reference to the table
Measurement = Base.classes.measurements
Station = Base.classes.stations

# Create our session (link) from Python to the DB
session = Session(engine)

In [3]:
latest_date= session.query(Measurement.date).order_by(Measurement.date.desc()).first()

In [9]:
start_date = dt.datetime(2017, 1, 1)
end_date = dt.datetime(2018, 1, 1)
calc_temps = session.query(func.max(Measurement.tobs), func.avg(Measurement.tobs), func.min(Measurement.tobs)).\
                           filter(Measurement.date > start_date).filter(Measurement.date < end_date).all()
calc_temps_start = session.query(func.max(Measurement.tobs), func.avg(Measurement.tobs), func.min(Measurement.tobs)).\
                           filter(Measurement.date > start_date).all() 

In [ ]:
#################################################
# Flask Setup
#################################################
app = Flask(__name__)


#################################################
# Flask Routes
#################################################

@app.route("/")
def welcome():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/startend<br/>"
        f"/api/v1.0/start<br/>"
    )
@app.route("/api/v1.0/precipitation")
def precipitation():
    """Return available last year precipitation data for Honolulu"""
    # Query date and tobs
    results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >'2014-04-30').filter(Measurement.date <'2015-10-30').\
    order_by(Measurement.date.desc()).filter(Measurement.station =='USC00511918').all()


    # Convert list of tuples into normal list
    precipitation_data = list(np.ravel(results))

    return jsonify(precipitation_data)

@app.route("/api/v1.0/stations")
def station():
    """Return a list of all stations names"""
    # Query all station names
    results = session.query(Station.station, Station.name).group_by(Station.station).all()  


    # Convert list of tuples into normal list
    stations_data = list(np.ravel(results))

    return jsonify(stations_data)



@app.route("/api/v1.0/tobs")
def tobs():
    """Return a tobs for previous year"""
    # Query all tobs for last year
    results = session.query(Measurement.station,Measurement.tobs).filter(Measurement.date <'2017-08-23').filter(Measurement.date >'2016-07-23').all()


    # Convert list of tuples into normal list
    tobs_data = list(np.ravel(results))

    return jsonify(tobs_data)


#To convert Date:
#Date 12 months ago
y = int(latest_date[0][:4])
m = int(latest_date[0][5:7])
d = int(latest_date[0][8:10])

start_date = dt.date(y, m, d) - dt.timedelta(days = 365)

@app.route("/api/v1.0/startend")
def tempstartend():

    calc_temps = session.query(func.max(Measurement.tobs), func.avg(Measurement.tobs), func.min(Measurement.tobs)).\
                           filter(Measurement.date > start_date).filter(Measurement.date < end_date).all()
    temp_ranges = list(np.ravel(calc_temps))

    return jsonify(temp_ranges)



@app.route("/api/v1.0/start")
def tempstart():

    calc_temps_start = session.query(func.max(Measurement.tobs), func.avg(Measurement.tobs), func.min(Measurement.tobs)).\
                           filter(Measurement.date > start_date).all()
    start_temp_ranges = list(np.ravel(calc_temps_start))
    
    return jsonify(start_temp_ranges)



if __name__ == '__main__':
    app.run(debug=False)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/May/2018 07:19:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2018 07:19:28] "GET /api/v1.0/start HTTP/1.1" 200 -
